- titanic ML 프로세스
0. 문제 정의
0-1. 목표 & 제출형식
1. 데이터 불러오기
2. 데이터 탐색(EDA)
3. 데이터 전처리
4. 특징 공학
5. 모델 선택 & 모델 학습
6. 모델 예측 & 모델 평가
7. 모델 튜닝
8. 모델 제출

0. 문제 정의
타이타닉호의 침몰은 역사상 가장 악명 높은 난파선 중 하나이다.

1912년 4월 15일, 처녀 항해 중, "침몰할 수 없는" RMS 타이타닉호가 빙산과 충돌한 후 침몰했다. 안타깝게도 탑승한 모든 사람을 위한 구명보트가 충분하지 않아 2224명의 승객과 승무원 중 1502명이 사망했습니다.

생존에 관련된 약간의 운의 요소가 있었지만, 어떤 집단은 다른 집단보다 살아남을 가능성이 더 높은 것으로 보인다.

이 도전에서, 우리는 당신에게 "어떤 종류의 사람들이 살아남을 가능성이 더 높습니까?"라는 질문에 답하는 예측 모델을 구축하도록 요청합니다. 승객 데이터 사용(예: 이름, 나이, 성별, 사회경제적 계층 등).

0-1. 목표 & 제출 형식

승객이 타이타닉 침몰에서 살아남았는지 아닌지를 예측하는 것은 당신의 일입니다.
테스트 세트의 각에 대해 변수에 대해 0 또는 1 값을 예측해야 합니다.

제출 파일 형식

정확히 418개의 항목과 헤더 행이 있는 csv 파일을 제출해야 합니다. 추가 열(PassingsingId 및 Survived 이상) 또는 행이 있는 경우 제출 시 오류가 표시됩니다.

파일은 정확히 2개의 열이 있어야 합니다.

PassengerId (모든 순서로 정렬)
생존 (당신의 이진 예측 포함: 생존은 1, 사망한 것은 0)
승객 Id, 생존
892,0
893,1
894,0
등등.


AI 학습의 중요한 개념들을 익혔습니다.

문제 정의: 타이타닉 승객 데이터를 이용해 생존 여부를 예측하는 분류 문제를 정의했습니다.
데이터 탐색 및 전처리:
train.csv와 test.csv 데이터를 불러오고 info(), head() 함수로 데이터의 전반적인 구조를 파악했습니다.
결측치가 있는 Age 컬럼을 중앙값으로 채우는 방법을 배웠고, 데이터 보존을 위해 새로운 컬럼(Age_fill)을 생성했습니다.
범주형 데이터인 Sex 컬럼을 숫자로 변환(Sex_chg)하는 방법을 익혔습니다.
특징 공학:
SibSp와 Parch 컬럼을 활용하여 새로운 의미 있는 특징인 FamilySize를 생성했습니다.
성별, 나이 그룹, 가족 규모가 생존율에 미치는 영향을 직접 분석하며 데이터의 숨겨진 패턴을 발견했습니다.
모델 선택 및 훈련:
분류 문제에 적합한 여러 모델들을 알아보고, 로지스틱 회귀 모델을 선택했습니다.
모델 학습에 필요한 X_train, y_train, X_test 데이터를 올바르게 정의했습니다.
Scikit-learn 라이브러리를 사용하여 로지스틱 회귀 모델을 훈련하는 코드를 작성했습니다.
예측 및 제출 파일 생성:
훈련된 모델로 test 데이터에 대한 생존 예측 값을 얻었고, 이를 캐글 제출 형식에 맞춰 CSV 파일로 저장하는 것까지 완료했습니다.
이 모든 과정을 혼자 힘으로, 그리고 저의 질문에 답하며 스스로 개념을 익히는 방식으로 진행하셨다는 점이 정말 대단합니다! LLM의 도움을 최소화하려는 목표를 성공적으로 달성하셨어요.

### 1. 데이터 불러오기

In [ ]:
import pandas as pd # 데이터 관리
import numpy as np  # 계산
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import cross_val_score, GridSearchCV

In [4]:
train = pd.read_csv(
    "/Users/ys/Library/Mobile Documents/com~apple~CloudDocs/Study/AI/Kaggle/titanic/data/train.csv"
)
test = pd.read_csv(
    "/Users/ys/Library/Mobile Documents/com~apple~CloudDocs/Study/AI/Kaggle/titanic/data/test.csv"
)

### 2. 데이터 탐색(EDA)

In [5]:
# 데이터 기본 정보 확인
print("=== 훈련 데이터 기본 정보 ===")
train.head()

# 결측치 비율 확인
print("=== 결측치 비율(%) ===")
train.isnull().mean() * 100

# 성별 별 생존율 - 남성 0.1889, 여성 0.74204
train.groupby("Sex")["Survived"].mean()

=== 훈련 데이터 기본 정보 ===
=== 결측치 비율(%) ===


Sex
female    0.742038
male      0.188908
Name: Survived, dtype: float64

In [ ]:


# 3.1. 성별 전처리 - 성별을 숫자로 변환
train['Sex_chg'] = train['Sex'].map({'male': 0, 'female': 1})
# 테스트 데이터에도 동일한 전처리 적용
test['Sex_chg'] = test['Sex'].map({'male': 0, 'female': 1})
# 2-2. 나이 구간 별 생존율 - 성인(18세 이상) 0.3611, 미성년 0.5398 
# 나이 결측치를 중앙값으로 대체(train, test 데이터 모두)
train['Age_fill'] = train['Age'].fillna(train['Age'].median())
test['Age_fill'] = test['Age'].fillna(test['Age'].median())
# 나이 구간을 성인(18세 이상)과 미성년(18세 미만)으로 나누기 
train['Age_Grp'] = np.where(train['Age_fill'] >= 18, 1, 0)
train.groupby('Age_Grp')['Survived'].mean()
test['Age_Grp'] = np.where(test['Age_fill'] >= 18, 1, 0)

# 3. 데이터 전처리

# 4. 특성 공학
# 4-1. 가족 규모와 생존율 관계 - 1인 0.303, 2인 0.553, 3인 0.578, 4인 0.724, 5인 0.2 ...
# 가족 규모를 계산하여 새로운 컬럼 생성
train['FamilySize'] = train['SibSp'] + train['Parch'] + 1
test['FamilySize'] = test['SibSp'] + test['Parch'] + 1
# 가족 규모 별 생존율
train.groupby('FamilySize')['Survived'].mean()

# 5. 모델 선택 & 모델 학습 - 로지스틱 회귀
# 5-1. 훈련 데이터 정의 - 독립변수(X)와 종속변수(y)
# 독립변수: 성별, 나이 구간, 가족 규모
# train 데이터에서 특성(X)과 타겟(y) 정의
X_train = train[['Sex_chg', 'Age_Grp', 'FamilySize', 'Pclass']]
y_train = train['Survived'] # 타겟은 보통 하나의 컬럼이므로 대괄호 하나로 선택
# test 데이터에서 특성(X) 정의
X_test = test[['Sex_chg', 'Age_Grp', 'FamilySize', 'Pclass']]
# 5-2. 로지스틱 회귀 모델 생성 및 학습
lm = LogisticRegression()
lm.fit(X_train, y_train)

# 6. 모델 예측 & 평가
# test 데이터에 대한 예측
lm.predict(X_test)
# 6-1. 예측 결과를 DataFrame으로 변환
submission = pd.DataFrame({
    'PassengerId' : test['PassengerId'],
    'Survived' : lm.predict(X_test)
})
# 6-2. 예측 결과를 CSV 파일로 저장
# submission.to_csv('/Users/rick/Library/Mobile Documents/com~apple~CloudDocs/Study/통계&빅데이터/Kaggle/titanic/submission_self.csv', index=False)
# 6-3. 모델 성능 평가
# 훈련 데이터에 대한 정확도 측정
# train_accuracy = lm.score(X_train, y_train)
# print(f"훈련 데이터 정확도: {train_accuracy:.4f}")
# 5-폴드 교차 검증 (k=5)
# from sklearn.model_selection import cross_val_score(상단 추가)
# cv=5는 데이터를 5개의 폴드로 나누어 5번 훈련 및 평가를 진행하라는 의미입니다.
# scoring='accuracy'는 평가 지표로 정확도를 사용하겠다는 의미입니다.
# cv_scores = cross_val_score(lm, X_train, y_train, cv=5, scoring='accuracy')
# print(f"교차 검증 정확도 (각 폴드): {cv_scores}")
# print(f"교차 검증 평균 정확도: {cv_scores.mean():.4f}")
# print(f"교차 검증 정확도 표준 편차: {cv_scores.std():.4f}")

# 7. 모델 튜닝 - 하이퍼파라미터 튜닝
# from sklearn.model_selection import GridSearchCV(상단 추가)
# 튜닝할 하이퍼파라미터의 값들을 딕셔너리 형태로 정의합니다.
# key는 하이퍼파라미터 이름, value는 시도해볼 값들의 리스트입니다.
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],  # C 값을 5가지 시도
    'solver': ['liblinear', 'lbfgs'] # solver를 2가지 시도
}
# GridSearchCV 객체 생성
# estimator: 튜닝할 모델 (여기서는 LogisticRegression)
# param_grid: 튜닝할 하이퍼파라미터와 값들의 딕셔너리
# cv: 교차 검증 폴드 수 (예: 5)
# scoring: 모델 평가 지표 (예: 'accuracy' 또는 'f1')
grid_search = GridSearchCV(estimator=LogisticRegression(max_iter=1000), # 충분한 반복 횟수 설정 
                           param_grid=param_grid,
                           cv=5, # 5-폴드 교차 검증
                           scoring='accuracy', # 정확도를 기준으로 최적 조합 찾기
                           n_jobs=-1) # 모든 코어 사용

grid_search.fit(X_train, y_train)
grid_search.predict(X_test)

submission_self_tune = pd.DataFrame({
    'PassengerId' : test['PassengerId'],
    'Survived' : grid_search.predict(X_test)
})
submission.to_csv('/Users/rick/Library/Mobile Documents/com~apple~CloudDocs/Study/통계&빅데이터/Kaggle/titanic/submission_self_tune2.csv', index=False)
# 훈련 데이터에 대한 정확도 측정
train_acc = grid_search.score(X_train, y_train)
print(f"훈련 데이터 정확도: {train_acc:.4f}")
# 5-폴드 교차 검증 (k=5)
# cv=5는 데이터를 5개의 폴드로 나누어 5번 훈련 및 평가를 진행하라는 의미입니다.
# scoring='accuracy'는 평가 지표로 정확도를 사용하겠다는 의미입니다.
cv_scores = cross_val_score(grid_search, X_train, y_train, cv=5, scoring='accuracy')
print(f"교차 검증 정확도 (각 폴드): {cv_scores}")
print(f"교차 검증 평균 정확도: {cv_scores.mean():.4f}")
print(f"교차 검증 정확도 표준 편차: {cv_scores.std():.4f}")

훈련 데이터 정확도: 0.8058
교차 검증 정확도 (각 폴드): [0.79888268 0.80898876 0.80898876 0.78089888 0.82022472]
교차 검증 평균 정확도: 0.8036
교차 검증 정확도 표준 편차: 0.0132


Exception ignored in: <function ResourceTracker.__del__ at 0x107485940>
Traceback (most recent call last):
  File "/Users/rick/miniforge3/envs/MathPython/lib/python3.12/multiprocessing/resource_tracker.py", line 77, in __del__
  File "/Users/rick/miniforge3/envs/MathPython/lib/python3.12/multiprocessing/resource_tracker.py", line 86, in _stop
  File "/Users/rick/miniforge3/envs/MathPython/lib/python3.12/multiprocessing/resource_tracker.py", line 111, in _stop_locked
ChildProcessError: [Errno 10] No child processes
Exception ignored in: <function ResourceTracker.__del__ at 0x109569940>
Traceback (most recent call last):
  File "/Users/rick/miniforge3/envs/MathPython/lib/python3.12/multiprocessing/resource_tracker.py", line 77, in __del__
  File "/Users/rick/miniforge3/envs/MathPython/lib/python3.12/multiprocessing/resource_tracker.py", line 86, in _stop
  File "/Users/rick/miniforge3/envs/MathPython/lib/python3.12/multiprocessing/resource_tracker.py", line 111, in _stop_locked
ChildProc